<a href="https://colab.research.google.com/github/good0855/daou-FITs-3-ParkSunhong/blob/%EC%8B%A4%EC%8A%B5/%5BCS%5D250109-%EC%8B%A4%EC%8A%B524-25%EB%B2%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# 실습 23 - 고객이탈 예측 모델 구축 실습
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [29]:
# 랜덤넘버 고정
np.random.seed(42)

In [30]:
# 고객수
n = 1000

In [31]:
data = {
    'customer_id': np.arange(1, n+1),
    'age' : np.random.randint(18, 70, n),
    'tenure': np.random.randint(1, 10, n),
    'balance' : np.random.randint(0, 25000, n),
    'num_products' : np.random.randint(1, 4, n),
    'has_credit_card': np.random.choice([0, 1], n),
    'is_active_member': np.random.choice([0, 1], n),
    'estimated_salary': np.random.uniform(30000, 150000, n),
    'churn': np.random.choice([0, 1], n, p=[0.8, 0.2])    # 고객 이탈 여부(20% 이탈)
}

In [32]:
# 데이터 생성
df = pd.DataFrame(data)
df.to_csv('customer_churn.csv', index=False)

print(df.head())

   customer_id  age  tenure  balance  num_products  has_credit_card  \
0            1   56       3    11563             1                1   
1            2   69       3    14051             3                0   
2            3   46       9    11022             1                1   
3            4   32       7     8234             3                1   
4            5   60       5    21829             1                0   

   is_active_member  estimated_salary  churn  
0                 1     122828.652208      0  
1                 1      35284.845527      0  
2                 0     113200.218448      0  
3                 0     107788.307686      0  
4                 1      57777.611510      0  


In [33]:
# 전처리 작업

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler



# 훈련, 테스트 데이터 분할

# 데이터 표준


In [34]:
# 결측치 확인
df.isnull().sum()


,0
customer_id,0
age,0
tenure,0
balance,0
num_products,0
has_credit_card,0
is_active_member,0
estimated_salary,0
churn,0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       1000 non-null   int64  
 1   age               1000 non-null   int64  
 2   tenure            1000 non-null   int64  
 3   balance           1000 non-null   int64  
 4   num_products      1000 non-null   int64  
 5   has_credit_card   1000 non-null   int64  
 6   is_active_member  1000 non-null   int64  
 7   estimated_salary  1000 non-null   float64
 8   churn             1000 non-null   int64  
dtypes: float64(1), int64(8)
memory usage: 70.4 KB


In [44]:
# 실습 24,25

In [37]:
# x 및 y 분리
y = df['churn']
x = df.drop('churn', axis=1)

# 범주형 변수 인코딩
x = pd.get_dummies(x, drop_first=True)

In [38]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [39]:
# 로지스틱 회귀분석
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model
model = LogisticRegression()
model.fit(x_train, y_train)

# predict
y_pred = model.predict(x_test)

# score
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

print("Classification Report")
print(classification_report(y_test, y_pred))

Accuracy: 0.835
Confusion Matrix
[[167   0]
 [ 33   0]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      1.00      0.91       167
           1       0.00      0.00      0.00        33

    accuracy                           0.83       200
   macro avg       0.42      0.50      0.46       200
weighted avg       0.70      0.83      0.76       200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
# 하이퍼파라미터 그리드
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
# 랜덤 포레스트 모델 튜닝
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(x_train, y_train)
# 최적 하이퍼파라미터
print(f'Best Parameters: {grid_search.best_params_}')
# 최적 모델 예측
best_rf_model = grid_search.best_estimator_
y_pred_tuned = best_rf_model.predict(x_test)
# 성능 평가
print(f'Accuracy: {accuracy_score(y_test, y_pred_tuned):.4f}')

Best Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy: 0.8350


In [41]:
# 모델 저장
import joblib
joblib.dump(best_rf_model, 'best_rf_model.pkl')

['best_rf_model.pkl']

In [42]:
# 모델을 불러서 사용하는 코드
loaded_model = joblib.load('best_rf_model.pkl')
# 모델로 예측
y_pred_loaded = loaded_model.predict(x_test)

In [43]:
y_pred_loaded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])